In [10]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import fashion_mnist, mnist
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping

In [11]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

x_train, x_test = x_train / 255.0, x_test / 255.0

x_train = x_train.reshape((x_train.shape[0], 28, 28, 1))
x_test = x_test.reshape((x_test.shape[0], 28, 28, 1))

y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

In [12]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = 0.2, random_state = 1234)

In [13]:
model = models.Sequential()
model.add(layers.experimental.preprocessing.RandomCrop(28, 28, input_shape=(28, 28, 1)))
model.add(layers.experimental.preprocessing.RandomFlip("horizontal"))
model.add(layers.Conv2D(64, (3, 3),
                        activation='relu',
                        input_shape=(28, 28, 1),
                        padding = "same")
         )
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3),
                        activation='relu',
                        padding = "same")
         )
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3),
                        activation='relu',
                        padding = "same")
         )
# model.add(layers.BatchNormalization())
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.4))
model.add(layers.Dense(10, activation='softmax'))

In [14]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 random_crop_1 (RandomCrop)  (None, 28, 28, 1)         0         
                                                                 
 random_flip_1 (RandomFlip)  (None, 28, 28, 1)         0         
                                                                 
 conv2d_3 (Conv2D)           (None, 28, 28, 64)        640       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 14, 14, 64)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 14, 14, 128)       73856     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 7, 7, 128)        0         
 2D)                                                  

In [15]:
model.compile(optimizer = "adam",
             loss = "categorical_crossentropy",
             metrics = ["accuracy"],
             )

early_stopping = EarlyStopping(monitor = "val_accuracy", patience = 10, restore_best_weights = True)

In [16]:
model.fit(x_train, y_train,
                  batch_size = 512,
                  epochs = 100,
                  validation_data=(x_val, y_val),
                  callbacks=[early_stopping],
         )

Epoch 1/100
94/94 [==============================] - 37s 385ms/step - loss: 0.8011 - accuracy: 0.7047 - val_loss: 0.4375 - val_accuracy: 0.8401
Epoch 2/100
94/94 [==============================] - 38s 405ms/step - loss: 0.4058 - accuracy: 0.8540 - val_loss: 0.3681 - val_accuracy: 0.8648
Epoch 3/100
94/94 [==============================] - 37s 398ms/step - loss: 0.3336 - accuracy: 0.8801 - val_loss: 0.3120 - val_accuracy: 0.8853
Epoch 4/100
94/94 [==============================] - 37s 398ms/step - loss: 0.2951 - accuracy: 0.8957 - val_loss: 0.2933 - val_accuracy: 0.8938
Epoch 5/100
94/94 [==============================] - 39s 409ms/step - loss: 0.2673 - accuracy: 0.9041 - val_loss: 0.2587 - val_accuracy: 0.9037
Epoch 6/100
54/94 [================>.............] - ETA: 15s - loss: 0.2468 - accuracy: 0.9085

KeyboardInterrupt: 

In [ ]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f'ACCURACY: {test_acc * 100:.2f}%')